In [1]:
import numpy as np
import pandas as pd

import matplotlib as plt
import seaborn as sns
import sqlalchemy as alch

from sqlalchemy import text
from sqlalchemy.sql import func
from sqlalchemy import inspect
from sqlalchemy_utils import database_exists, create_database

# Cursory Analysis

In [2]:
df = pd.read_csv('DATA\casestudy.csv')

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isna().sum
df.dtypes

In [ ]:
#df['Unnamed: 0'].value_counts() # <--- index
#df['net_revenue'].value_counts()
df['year'].value_counts()

# SQL Setup

In [3]:
engine = alch.create_engine('sqlite:///db.sqlite')
if not database_exists(engine.url):
    create_database(engine.url)
    
print(database_exists(engine.url))

True


In [4]:
connection = engine.connect()
metadata = alch.MetaData()

In [5]:
table_name = 'orders'
df.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=False,
)

In [6]:
inspector = inspect(engine)
inspector.get_columns('orders')

[{'name': 'Unnamed: 0',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'customer_email',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'net_revenue',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'year',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

# Queries

In [ ]:
# Verifying dataframe -> database 
statement='SELECT * FROM orders'
with engine.connect() as con:

    rs = con.execute(statement)

    for row in rs:
        print(row)

In [7]:
conn = engine.connect()

In [8]:
# question 0,0
q0_0 = text("SELECT SUM(net_revenue)"
            " FROM orders"
            " WHERE year=:x")

a0_0 = conn.execute(q0_0, {"x":2015}).fetchall()
print(a0_0)

[(29036749.18999953,)]


In [9]:
# question 0,1
# same as q0,0

In [10]:
# question 0,2
# N/A : first year

In [11]:
# question 0,3
# N/A : first year

In [12]:
# question 0,4
# same as 0,1

In [13]:
# question 0,5
# N/A : first year

In [14]:
# question 0,6
q0_6 = text("SELECT COUNT(*) FROM orders WHERE year=:x")
a0_6 = conn.execute(q0_6, {"x":2015}).fetchall()
print(a0_6)

[(231294,)]


In [15]:
# question 0,7
# N/A : first year

In [16]:
# question 0,8
# same as 0,6

In [17]:
# question 0,9
# N/A : first year

In [18]:
# question 1,0
q1_0 = text("SELECT SUM(net_revenue)"
            " FROM orders"
            " WHERE year=:x")

a1_0 = conn.execute(q1_0, {"x":2016}).fetchall()
print(a1_0)

[(25730943.58999988,)]


In [19]:
# question 1,1
q1_1 = text("SELECT SUM(net_revenue) "
            "FROM orders "
            "WHERE year=:x "
            "AND customer_email NOT IN (SELECT customer_email FROM orders WHERE year=:y)")

a1_1 = conn.execute(q1_1,{"x":2016,"y":2015}).fetchall()
print(a1_1)

[(18245491.010000203,)]


In [20]:
# question 1,2
t1 =    ("SELECT SUM(net_revenue) "
        "FROM orders "
        "WHERE year=:x "
        "AND customer_email IN (SELECT customer_email FROM orders WHERE year=:y)")

t2 =   ("SELECT SUM(net_revenue) "
        "FROM orders "
        "WHERE year=:y "
        "AND customer_email IN (SELECT customer_email FROM orders WHERE year=:x)")

d1 = conn.execute(t1,{"x":2016,"y":2015}).fetchall()
d2 = conn.execute(t2,{"x":2016,"y":2015}).fetchall()

print(d1[0])
print(d2[0])

a1_2 = 7485452.5800000075 - 7465117.120000009
print(a1_2)

(7485452.5800000075,)
(7465117.120000009,)
20335.4599999981


In [21]:
# question 1,3 : rev(existing customers)
q1_3a = "SELECT SUM(net_revenue) FROM orders WHERE year=:y"
q1_3b = "SELECT SUM(net_revenue) FROM orders WHERE year=:x"

print(conn.execute(q1_3a,{"x":2016,"y":2015}).fetchall())
print(conn.execute(q1_3b,{"x":2016,"y":2015}).fetchall())

a1_3 = 29036749.18999953 - 25730943.58999988
print(a1_3)

[(29036749.18999953,)]
[(25730943.58999988,)]
3305805.5999996476


In [22]:
# question 1,4
q1_4 = "SELECT SUM(net_revenue) FROM orders WHERE year=:x AND customer_email IN (SELECT customer_email FROM orders WHERE year=:y)"

a1_4 = conn.execute(q1_4,{"x":2016,"y":2015}).fetchall()
print(a1_4)

[(7485452.5800000075,)]


In [23]:
# question 1,5
# same as 1,2b
a1_5 = 7465117.120000009
print(a1_5)

7465117.120000009


In [24]:
# question 1,6
q1_6 = "SELECT COUNT(*) FROM orders WHERE year=:x"

a1_6a = conn.execute(q1_6,{"x":2016,"y":2015}).fetchall()
print(a1_6a)
print(a0_6)
a1_6 = 204646+231294
print(a1_6)

[(204646,)]
[(231294,)]
435940


In [25]:
# question 1,7
# same as 0_6
a1_7 = a0_6
print(a1_7)

[(231294,)]


In [26]:
# question 1,8
q1_8 = ("SELECT COUNT(*) FROM orders WHERE year=:x "
       "AND customer_email NOT IN (SELECT customer_email WHERE year=:y)")

a1_8 = conn.execute(q1_8,{"x":2016,"y":2015}).fetchall()
print(a1_8)

[(204646,)]


In [27]:
# question 1,9
q1_9 = ("SELECT COUNT(*) FROM orders WHERE year=:x "
        "AND customer_email NOT IN (SELECT customer_email WHERE year=:y)")

a1_9 = conn.execute(q1_9,{"x":2015,"y":2016}).fetchall()
print(a1_9)

[(231294,)]


In [28]:
# question 2,0
q2_0 = "SELECT SUM(net_revenue) FROM orders WHERE year=:x"

a2_0 = conn.execute(q2_0,{"x":2017}).fetchall()

print(a2_0)

[(31417495.02999995,)]


In [29]:
# question 2,1
q2_1 = ("SELECT SUM(net_revenue) FROM orders WHERE year=:x "
       "AND customer_email NOT IN (SELECT customer_email FROM orders WHERE year=:y)")

a2_1 = conn.execute(q2_1,{"x":2017,"y":2016}).fetchall()

print(a2_1)

[(28776235.04,)]


In [30]:
# question 2,2
t1 =    ("SELECT SUM(net_revenue) "
        "FROM orders "
        "WHERE year=:x "
        "AND customer_email IN (SELECT customer_email FROM orders WHERE year=:y)")

t2 =   ("SELECT SUM(net_revenue) "
        "FROM orders "
        "WHERE year=:y "
        "AND customer_email IN (SELECT customer_email FROM orders WHERE year=:x)")

d1 = conn.execute(t1,{"x":2017,"y":2016}).fetchall()
d2 = conn.execute(t2,{"x":2017,"y":2016}).fetchall()

print(d1[0])
print(d2[0])

a2_2 = 2641259.990000008 - 2620648.6499999906
print(a2_2)

(2641259.990000008,)
(2620648.6499999906,)
20611.340000017546


In [31]:
# question 2,3
q2_3a = "SELECT SUM(net_revenue) FROM orders WHERE year=:y"
q2_3b = "SELECT SUM(net_revenue) FROM orders WHERE year=:x"

print(conn.execute(q2_3a,{"x":2017,"y":2016}).fetchall())
print(conn.execute(q2_3b,{"x":2017,"y":2016}).fetchall())

a2_3 = 25730943.58999988 - 31417495.02999995
print(a2_3)

[(25730943.58999988,)]
[(31417495.02999995,)]
-5686551.440000068


In [32]:
# question 2,4
q2_4 = "SELECT SUM(net_revenue) FROM orders WHERE year=:x AND customer_email IN (SELECT customer_email FROM orders WHERE year=:y)"

a2_4 = conn.execute(q2_4,{"x":2017,"y":2016}).fetchall()
print(a2_4)

[(2641259.990000008,)]


In [33]:
# question 2,5
q2_5 = "SELECT SUM(net_revenue) FROM orders WHERE year=:x AND customer_email IN (SELECT customer_email FROM orders WHERE year=:y)"

a2_5 = conn.execute(q2_5,{"x":2016,"y":2017}).fetchall()
print(a2_5)

[(2620648.6499999906,)]


In [34]:
# question 2,6 : interpreted the question as total customers in current year cumulative with past years
q2_6 = "SELECT COUNT(*) FROM orders WHERE year=:x"

a2_6a = conn.execute(q2_6,{"x":2016,"y":2015}).fetchall()

print(a2_6a)
print(a1_6)

a2_6 = 204646 + 435940
print(a2_6)

[(204646,)]
435940
640586


In [35]:
# question 2,7
# same as 1,6

In [36]:
# question 2,8
q2_8 = ("SELECT COUNT(*) FROM orders WHERE year=:x "
       "AND customer_email NOT IN (SELECT customer_email WHERE year=:y)")

a2_8 = conn.execute(q1_8,{"x":2017,"y":2016}).fetchall()
print(a2_8)

[(249987,)]


In [37]:
# question 2,9 
q2_9 = ("SELECT COUNT(*) FROM orders WHERE year=:x "
        "AND customer_email NOT IN (SELECT customer_email WHERE year=:y)")

a2_9 = conn.execute(q1_9,{"x":2016,"y":2017}).fetchall()
print(a2_9)

[(204646,)]
